In [1]:
import requests
import pandas as pd
import json
pd.set_option('display.max_columns', None)

In [2]:
# Realizamos una petición al servicio rest y comprobamos que se ha realizado correctamente (código de estado = 200)
request = requests.get("https://sedeaplicaciones.minetur.gob.es/ServiciosRESTCarburantes/PreciosCarburantes/EstacionesTerrestres/")
print(request.status_code)

200


In [3]:
# Formateamos la respuesta en formato JSON
json_data = request.json()

In [4]:
# Almacenamos por separado los valores correspondientes a la fecha de la petición y al listado de los precios de las estaciones de servicio
fecha = json_data["Fecha"]
precios = json_data["ListaEESSPrecio"]
print(fecha)

08/04/2022 15:09:59


In [5]:
# Creamos un dataframe con los precios y comprobamos que se han obtenido correctamente
df = pd.json_normalize(precios)
print(df.shape)
df.head()

(11654, 32)


,C.P.,Dirección,Horario,Latitud,Localidad,Longitud (WGS84),Margen,Municipio,Precio Biodiesel,Precio Bioetanol,Precio Gas Natural Comprimido,Precio Gas Natural Licuado,Precio Gases licuados del petróleo,Precio Gasoleo A,Precio Gasoleo B,Precio Gasoleo Premium,Precio Gasolina 95 E10,Precio Gasolina 95 E5,Precio Gasolina 95 E5 Premium,Precio Gasolina 98 E10,Precio Gasolina 98 E5,Precio Hidrogeno,Provincia,Remisión,Rótulo,Tipo Venta,% BioEtanol,% Éster metílico,IDEESS,IDMunicipio,IDProvincia,IDCCAA
0,02250,"AVENIDA CASTILLA LA MANCHA, 26",L-D: 07:00-22:00,"39,211417",ABENGIBRE,"-1,539167",D,Abengibre,,,,,,"1,829","1,340",,,"1,799",,,,,ALBACETE,dm,Nº 10.935,P,"0,0","0,0",4375,52,02,07
1,02152,"CR CM-332, 46,4",L-D: 7:00-23:00,"39,100389",ALATOZ,"-1,346083",I,Alatoz,,,,,,"1,950",,"2,000",,"1,869",,,"2,000",,ALBACETE,dm,REPSOL,P,"0,0","0,0",5122,53,02,07
2,02001,CALLE PRINCIPE DE ASTURIAS (POLÍGONO DE ROMICA...,L-D: 06:00-21:30,"39,054694",ALBACETE,"-1,832000",I,Albacete,,,,,,"1,859",,"1,949",,"1,799",,,"1,941",,ALBACETE,OM,BP ROMICA,P,"0,0","0,0",12054,54,02,07
3,02001,"AVENIDA 1º DE MAYO, S/N",L-S: 08:00-22:00; D: 09:00-21:00,"38,985667",ALBACETE,"-1,868500",N,Albacete,,,,,,"1,859",,"1,889",,"1,799",,,"1,899",,ALBACETE,dm,CARREFOUR,P,"0,0","0,0",10765,54,02,07
4,02001,"CALLE FEDERICO GARCIA LORCA, 5",L-S: 05:00-23:00,"38,997722",ALBACETE,"-1,846361",I,Albacete,,,,,,"1,879",,,,,,,,,ALBACETE,dm,CEPSA,R,"0,0","0,0",4438,54,02,07


In [6]:
print(df.columns) # Mostramos las columnas originales

Index(['C.P.', 'Dirección', 'Horario', 'Latitud', 'Localidad',
       'Longitud (WGS84)', 'Margen', 'Municipio', 'Precio Biodiesel',
       'Precio Bioetanol', 'Precio Gas Natural Comprimido',
       'Precio Gas Natural Licuado', 'Precio Gases licuados del petróleo',
       'Precio Gasoleo A', 'Precio Gasoleo B', 'Precio Gasoleo Premium',
       'Precio Gasolina 95 E10', 'Precio Gasolina 95 E5',
       'Precio Gasolina 95 E5 Premium', 'Precio Gasolina 98 E10',
       'Precio Gasolina 98 E5', 'Precio Hidrogeno', 'Provincia', 'Remisión',
       'Rótulo', 'Tipo Venta', '% BioEtanol', '% Éster metílico', 'IDEESS',
       'IDMunicipio', 'IDProvincia', 'IDCCAA'],
      dtype='object')


In [7]:
# Creamos un diccionario para sustituir el identificador de cada Comunidad Autónoma por su valor original
CCAA = {
    "01": "ANDALUCÍA", "02": "ARAGÓN", "03": "PRINCIPADO DE ASTURIAS", "04": "BALEARES", "05": "CANARIAS", "06": "CANTABRIA", "07": "CASTILLA LA MANCHA", "08": "CASTILLA Y LEÓN", "09": "CATALUÑA", "10": "COMUNIDAD VALENCIANA", "11": "EXTREMADURA", "12": "GALICIA", "13": "COMUNIDAD DE MADRID", "14": "REGIÓN DE MURCIA", "15": "COMUNIDAD FORAL DE NAVARRA", "16": "PAIS VASCO", "17": "LA RIOJA", "18": "CIUDAD AUTÓNOMA DE CEUTA", "19": "CIUDAD AUTÓNOMA DE MELILLA",
}

In [8]:
# Creamos un DataFrame únicamente con las columnas que nos interesan persistir
eess_df = df[["IDEESS", "Dirección", "Horario", "Rótulo", "Margen", "Tipo Venta", "Remisión", "Latitud", "Longitud (WGS84)", "Municipio", "C.P.", "Provincia", "IDCCAA"]]

# Sustituimos los identificadores de cada Comunidad Autónoma por su valor original
eess_df = eess_df.replace({"IDCCAA": CCAA}) 
# Renombramos la columna para que el título se corresponda con su contenido
eess_df.rename(columns={"IDCCAA": "CCAA"}, inplace=True)

print(eess_df.shape)
eess_df.head()

(11654, 13)


,IDEESS,Dirección,Horario,Rótulo,Margen,Tipo Venta,Remisión,Latitud,Longitud (WGS84),Municipio,C.P.,Provincia,CCAA
0,4375,"AVENIDA CASTILLA LA MANCHA, 26",L-D: 07:00-22:00,Nº 10.935,D,P,dm,"39,211417","-1,539167",Abengibre,02250,ALBACETE,CASTILLA LA MANCHA
1,5122,"CR CM-332, 46,4",L-D: 7:00-23:00,REPSOL,I,P,dm,"39,100389","-1,346083",Alatoz,02152,ALBACETE,CASTILLA LA MANCHA
2,12054,CALLE PRINCIPE DE ASTURIAS (POLÍGONO DE ROMICA...,L-D: 06:00-21:30,BP ROMICA,I,P,OM,"39,054694","-1,832000",Albacete,02001,ALBACETE,CASTILLA LA MANCHA
3,10765,"AVENIDA 1º DE MAYO, S/N",L-S: 08:00-22:00; D: 09:00-21:00,CARREFOUR,N,P,dm,"38,985667","-1,868500",Albacete,02001,ALBACETE,CASTILLA LA MANCHA
4,4438,"CALLE FEDERICO GARCIA LORCA, 5",L-S: 05:00-23:00,CEPSA,I,R,dm,"38,997722","-1,846361",Albacete,02001,ALBACETE,CASTILLA LA MANCHA


In [9]:
# Almacenamos los datos
eess_df.to_csv("../data/EESS.csv", sep=";", encoding="utf-8", index=False, header=True)